In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [3]:
t1=time.time()
G=build_graph(cursor)
t2=time.time()
print(t2-t1)

0.9450118541717529


In [4]:
t1=time.time()
cmd="SELECT osm_id,contour_line_id,ST_asText(geometry) AS geometry FROM %s WHERE contour_line_id IS NOT NULL"%osm_nodes_table_name
cursor.execute(cmd)
nodes_data=pd.DataFrame(cursor.fetchall())
nodes_data['geometry']=nodes_data['geometry'].apply(lambda pt:loads(pt))
nodes_data=nodes_data.set_index('osm_id',drop=False)
nodes_data=gpd.GeoDataFrame(nodes_data,geometry='geometry',crs='epsg:4326')
nodes_data['old_geometry']=nodes_data['geometry'].copy()
crs=nodes_data.estimate_utm_crs()
nodes_data=nodes_data.to_crs(crs)
nodes_data['depth']=nodes_data['contour_line_id'].apply(lambda contour_line_id:G.nodes()[contour_line_id]['depth'])
t2=time.time()
print(t2-t1)

0.49240541458129883


In [5]:
len(nodes_data)

17433

In [6]:
max_nb_neighbors=float('inf')
t1=time.time()
nodes_data['elevation']=None
contour_line_ids=nodes_data['contour_line_id'].unique()
contour_lines_ids=sorted(contour_line_ids,key=lambda contour_line_id:len(list(G.neighbors(contour_line_id))))
contour_lines_ids=contour_lines_ids[::-1]
print('sorted')
for k,root_contour_line_id in enumerate(contour_lines_ids):
    if (k+1)%10==0:
        print('%i/%i'%(k+1,len(contour_line_ids)))
    neighbor_contour_line_ids=list(G.neighbors(root_contour_line_id))
    print('%i neighbors'%len(neighbor_contour_line_ids))
    if 0<len(neighbor_contour_line_ids)<max_nb_neighbors:
        t3=time.time()
        local_contours_line_ids=[root_contour_line_id]+neighbor_contour_line_ids
        local_contour_data=get_nodes_data(cursor,local_contours_line_ids)
        local_contour_data=local_contour_data.to_crs(crs)
        local_contour_data=local_contour_data.set_index('id')
        t4=time.time()
        print('setting up df took %f'%(t4-t3))
        local_nodes_data=nodes_data[nodes_data.contour_line_id==root_contour_line_id].copy()
        local_nodes_data['elevation']=local_nodes_data['geometry'].apply(lambda osm_pt:get_osm_node_elevation(osm_pt,local_contour_data))
        nodes_data.loc[nodes_data.contour_line_id==root_contour_line_id,'elevation']=local_nodes_data.loc[:,'elevation']
    
        t5=time.time()
        print('finding affine basis took %f'%(t5-t4))
        print('intermediate time %f'%(t5-t1))
t2=time.time()
print(t2-t1)

sorted
607 neighbors
setting up df took 0.696422


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 35.487742
intermediate time 36.186804
228 neighbors
setting up df took 0.845675


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 29.925716
intermediate time 66.958275
205 neighbors
setting up df took 0.923965


...

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


setting up df took 0.016549
finding affine basis took 0.003103
intermediate time 635.600824
1 neighbors
setting up df took 0.007577
finding affine basis took 0.002668
intermediate time 635.611092
1 neighbors
setting up df took 0.007451
finding affine basis took 0.002642
intermediate time 635.621209
1 neighbors
setting up df took 0.007365
finding affine basis took 0.003189
intermediate time 635.631788
1 neighbors
setting up df took 0.007715
finding affine basis took 0.002669
intermediate time 635.642195
1 neighbors
setting up df took 0.036067
finding affine basis took 0.003412
intermediate time 635.681699
1 neighbors
setting up df took 0.034972
finding affine basis took 0.003423
intermediate time 635.720119
1 neighbors
setting up df took 0.004926
finding affine basis took 0.002536
intermediate time 635.727620
1 neighbors
setting up df took 0.004631
finding affine basis took 0.002518
intermediate time 635.734792
1 neighbors
setting up df took 0.006760
finding affine basis took 0.002727
i

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


setting up df took 0.005291
finding affine basis took 0.003253
intermediate time 635.807796
1 neighbors
setting up df took 0.005006
finding affine basis took 0.003116
intermediate time 635.815948
1 neighbors
setting up df took 0.005248
finding affine basis took 0.002723
intermediate time 635.823948
1 neighbors
setting up df took 0.055017
finding affine basis took 0.004662
intermediate time 635.883653
830/971
1 neighbors
setting up df took 0.062182
finding affine basis took 0.005600
intermediate time 635.951467
1 neighbors
setting up df took 0.005762
finding affine basis took 0.002550
intermediate time 635.959805
1 neighbors
setting up df took 0.004367
finding affine basis took 0.002777
intermediate time 635.966973
1 neighbors
setting up df took 0.004123
finding affine basis took 0.002471
intermediate time 635.973607
1 neighbors
setting up df took 0.006153
finding affine basis took 0.003788
intermediate time 635.983571
1 neighbors
setting up df took 0.005719
finding affine basis took 0.

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


In [10]:
nodes_data=nodes_data[nodes_data.elevation==nodes_data.elevation]
diffs=nodes_data['elevation']-nodes_data['contour_line_id'].apply(lambda node:G.nodes()[node]['elevation'])
np.min(diffs),np.mean(diffs),np.median(diffs),np.max(diffs)
                                                                  

(-0.4596825749848179, 2.514590478087124, 2.5335205308406734, 4.999915106494939)

In [11]:
UPDATE_ELEVATION=True
if UPDATE_ELEVATION:
    t1=time.time()
    cmd="""CREATE TEMPORARY TABLE temp (
    `osm_id` BIGINT NOT NULL,
    `elevation` FLOAT NOT NULL

    );"""
    cursor.execute(cmd)

    cmd="INSERT INTO temp(`osm_id`,`elevation`) VALUES %s"%',\n'.join(['(%i,%f)'%(osm_id,row['elevation']) for osm_id,row in nodes_data.iterrows()])
    cursor.execute(cmd)

    cmd=""" UPDATE %s JOIN temp
    ON %s.osm_id=temp.osm_id
    SET %s.elevation=temp.elevation

    ;"""%(osm_nodes_table_name,osm_nodes_table_name,osm_nodes_table_name)
    cursor.execute(cmd)
    
    cmd="DROP TABLE temp"
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)
2

0.8259708881378174


2